<a href="https://colab.research.google.com/github/i-m-vivek/Deep-Learning-Models/blob/master/Image_to_Vector_EyeGaze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook I have used all the data available.
<br>
Train -> 1280 X 100
<br>
Val -> 1280 X 50

In [1]:
from IPython.display import clear_output

In [1]:
!nvidia-smi

Tue Jun 16 19:04:44 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Image to Vector

**Task :** Given an image only we need to predict the gazing vector.
**Dataset :** We used the train set of OpenEDS Challenge for each sequence 10 frames are sampled at an interval of 10.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [8]:
from IPython.display import clear_output
!unrar x "/content/drive/My Drive/EyeGaze/data/Complete Data Resized/resized_complete_train.rar"
!unrar x "/content/drive/My Drive/EyeGaze/data/Complete Data Resized/resized_complete_val.rar"
clear_output()

## Dataloaders

In [9]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

import tqdm
import pandas as pd 
import cv2 as cv
import numpy as np 
from PIL import Image
from skimage import io, transform
import time
import os
import copy

In [10]:
class EyeGazeDataset(Dataset):
    def __init__(self, root_dir, df, transform= None):
        self.root_dir = root_dir 
        self.df = df.sample(frac= 1)
        self.transform = transform
    def __getitem__(self, idx):
        vector  = np.array(self.df.iloc[idx, 1:] , dtype= np.float64)
        # img = io.imread(os.path.join(self.root_dir, self.df.iloc[idx, 0]))
        img_folder, image_name = self.df.iloc[idx, 0].split("_")
        if len(image_name) == 1:
            image_name = "00" + image_name + ".png"
        elif len(image_name) == 2:
            image_name = "0" + image_name +".png"
        img_path = os.path.join(os.path.join(self.root_dir, img_folder), image_name)
        img = cv.imread(img_path)
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        if self.transform is not None:
            img = self.transform(img)
        return img, torch.tensor(vector)
    def __len__(self):
        return len(self.df)

In [11]:
train = pd.read_csv("/content/drive/My Drive/EyeGaze/data/Complete Data Resized/all_data_train.csv")
val = pd.read_csv("/content/drive/My Drive/EyeGaze/data/Complete Data Resized/all_data_val.csv")

In [12]:
BS = 32

In [13]:
composed_transform = transforms.Compose([
   transforms.ToTensor(),
   transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),])

train_dataset= EyeGazeDataset("/content/train/sequences", train, composed_transform)
val_dataset= EyeGazeDataset("/content/val/sequences", val, composed_transform)

image_datasets = {"train": train_dataset ,
                  "val":val_dataset }
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=BS,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [14]:
dataset_sizes

{'train': 128000, 'val': 64000}

## Loss Function

In [15]:
def acos_loss(pred, true):
    """
    Expected shape for true and pred is (m, 3)
    where m is the batchsize and 3 are the x, y, z coordinates
    """
    bs = pred.size(0)
    batch_dot_product = torch.bmm(pred.view(bs, 1, 3), true.view(bs, 3, 1)).reshape(bs, )
    norm_true= torch.norm(true, p= 2, dim= -1)
    norm_pred= torch.norm(pred, p= 2, dim= -1)

    loss = torch.mean(torch.acos(batch_dot_product/(norm_true*norm_pred)))
    return loss

## Model

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### ResNet 18

In [17]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)
model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth


In [ ]:
!nvidia-smi

Tue Jun 16 16:30:47 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    34W / 250W |    773MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [19]:
loaded = torch.load("/content/drive/My Drive/EyeGaze/checkpoint_resnet18_acosLoss_25epochs_complete_data.pth")

In [20]:
loaded.keys()

dict_keys(['Total Epoch', 'This Epoch: ', 'loss', 'loss_function', 'Model', 'BS', 'train_data & val_data', 'model', 'optimizer', 'scheduler'])

In [21]:
model.load_state_dict(loaded["model"])

<All keys matched successfully>

In [26]:
optimizer = optim.Adam(model.parameters(), lr=0.003, weight_decay=.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 8, gamma=.1)

In [27]:
scheduler.load_state_dict(loaded["scheduler"])
optimizer.load_state_dict(loaded["optimizer"])

In [28]:
num_epochs = 15

In [ ]:
since = time.time()

best_model_wts = copy.deepcopy(model.state_dict())
best_loss = 99999

train_loss = []
val_loss = []

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Set model to training mode
        else:
            model.eval()   # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        num_steps = 0

        # Iterate over data.
        for inputs, labels in tqdm.notebook.tqdm(dataloaders[phase]):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                # if criterion is not None:
                #     loss = criterion(outputs.type(torch.DoubleTensor), labels.type(torch.DoubleTensor))
                # else:
                loss = acos_loss(outputs.type(torch.DoubleTensor), labels.type(torch.DoubleTensor))
                    
                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # statistics
            if phase == "train":
                train_loss.append(loss.item())
            elif phase == "val":
                val_loss.append(loss.item())

            running_loss += loss.item()*inputs.size(0)
            num_steps += 1 
            if num_steps%800 == 0:
                print("**"*10)
                print("{} Epoch: {} Loss: {:.4f}".format(phase, epoch, loss.item()))
        

        epoch_loss = running_loss / dataset_sizes[phase]
        if phase == 'val' and scheduler != None:
            scheduler.step()

        print('{} Loss: {:.4f}'.format(
            phase, epoch_loss))

        # deep copy the model
        if phase == 'val' and epoch_loss < best_loss:
            best_loss = epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save({
                "Total Epoch": num_epochs, 
                "This Epoch: ": epoch+1,
                "loss":best_loss,
                "loss_function": "acos_loss",
                "Model": "Only ResNet18 for Image to Vector Prediction with Complete Resized Dataset", 
                "BS": BS,
                "train_data & val_data": "Complete Data",
                "model": model.state_dict(), 
                "optimizer":optimizer.state_dict() ,
                "scheduler": scheduler.state_dict(),
            }, "/content/drive/My Drive/EyeGaze/checkpoint_resnet18_acosLoss_25epochs_complete_data.pth", )

    print()

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))
print('Best val Loss: {:4f}'.format(best_loss))

# load best model weights
model.load_state_dict(best_model_wts)

Epoch 0/24
----------


********************
train Epoch: 0 Loss: 0.0757
********************
train Epoch: 0 Loss: 0.0663
********************
train Epoch: 0 Loss: 0.0462
********************
train Epoch: 0 Loss: 0.0635
********************
train Epoch: 0 Loss: 0.0655

train Loss: 0.0659


********************
val Epoch: 0 Loss: 0.0952
********************
val Epoch: 0 Loss: 0.1002

val Loss: 0.1119

Epoch 1/24
----------


********************
train Epoch: 1 Loss: 0.0556
********************
train Epoch: 1 Loss: 0.0358
********************
train Epoch: 1 Loss: 0.0419
********************
train Epoch: 1 Loss: 0.0388
********************
train Epoch: 1 Loss: 0.0422

train Loss: 0.0427


********************
val Epoch: 1 Loss: 0.0843
********************
val Epoch: 1 Loss: 0.0814

val Loss: 0.0879

Epoch 2/24
----------


...

********************
train Epoch: 9 Loss: 0.0135
********************
train Epoch: 9 Loss: 0.0191
********************
train Epoch: 9 Loss: 0.0124
********************
train Epoch: 9 Loss: 0.0127
********************
train Epoch: 9 Loss: 0.0181

train Loss: 0.0166


********************
val Epoch: 9 Loss: 0.0731
********************
val Epoch: 9 Loss: 0.0902

val Loss: 0.0737

Epoch 10/24
----------


********************
train Epoch: 10 Loss: 0.0143
********************
train Epoch: 10 Loss: 0.0180
********************
train Epoch: 10 Loss: 0.0170
********************
train Epoch: 10 Loss: 0.0151
********************
train Epoch: 10 Loss: 0.0165

train Loss: 0.0153


********************
val Epoch: 10 Loss: 0.0743
********************
val Epoch: 10 Loss: 0.0580

val Loss: 0.0741

Epoch 11/24
----------


********************
train Epoch: 11 Loss: 0.0117
********************
train Epoch: 11 Loss: 0.0142


In [ ]:
since = time.time()

best_model_wts = copy.deepcopy(model.state_dict())
best_loss = 99999

train_loss = []
val_loss = []

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Set model to training mode
        else:
            model.eval()   # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        num_steps = 0

        # Iterate over data.
        for inputs, labels in tqdm.notebook.tqdm(dataloaders[phase]):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                # if criterion is not None:
                #     loss = criterion(outputs.type(torch.DoubleTensor), labels.type(torch.DoubleTensor))
                # else:
                loss = acos_loss(outputs.type(torch.DoubleTensor), labels.type(torch.DoubleTensor))
                    
                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # statistics
            if phase == "train":
                train_loss.append(loss.item())
            elif phase == "val":
                val_loss.append(loss.item())

            running_loss += loss.item()*inputs.size(0)
            num_steps += 1 
            if num_steps%800 == 0:
                print("**"*10)
                print("{} Epoch: {} Loss: {:.4f}".format(phase, epoch, loss.item()))
        

        epoch_loss = running_loss / dataset_sizes[phase]
        if phase == 'val' and scheduler != None:
            scheduler.step()

        print('{} Loss: {:.4f}'.format(
            phase, epoch_loss))

        # deep copy the model
        if phase == 'val' and epoch_loss < best_loss:
            best_loss = epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save({
                "Total Epoch": num_epochs+ 10, 
                "This Epoch: ": epoch+1 + 10,
                "loss":best_loss,
                "loss_function": "acos_loss",
                "Model": "Only ResNet18 for Image to Vector Prediction with Complete Resized Dataset", 
                "BS": BS,
                "train_data & val_data": "Complete Data",
                "model": model.state_dict(), 
                "optimizer":optimizer.state_dict() ,
                "scheduler": scheduler.state_dict(),
            }, "/content/drive/My Drive/EyeGaze/checkpoint_resnet18_acosLoss_25epochs_complete_data.pth", )

    torch.save({
                "Total Epoch": num_epochs + 10, 
                "This Epoch: ": epoch+1+ 10,
                "loss":best_loss,
                "loss_function": "acos_loss",
                "Model": "Only ResNet18 for Image to Vector Prediction with Complete Resized Dataset", 
                "BS": BS,
                "train_data & val_data": "Complete Data",
                "model": model.state_dict(), 
                "optimizer":optimizer.state_dict() ,
                "scheduler": scheduler.state_dict(),
            }, "/content/drive/My Drive/EyeGaze/checkpoint_resnet18_acosLoss_25epochs_complete_data_every_epoch.pth", )

    print()

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))
print('Best val Loss: {:4f}'.format(best_loss))

# load best model weights
model.load_state_dict(best_model_wts)

Epoch 0/14
----------


********************
train Epoch: 0 Loss: 0.0155
********************
train Epoch: 0 Loss: 0.0141
********************
train Epoch: 0 Loss: 0.0171
********************
train Epoch: 0 Loss: 0.0225
********************
train Epoch: 0 Loss: 0.0135

train Loss: 0.0153


In [5]:
loaded.keys()

dict_keys(['Total Epoch', 'This Epoch: ', 'loss', 'loss_function', 'Model', 'BS', 'train_data & val_data', 'model', 'optimizer', 'scheduler'])

In [7]:
loaded["This Epoch: "]

10

In [ ]:
torch.save({
    "Epoch": num_epochs, 
    "loss":best_loss,
    "loss_function": "acos_loss",
    "Model": "Only ResNet18 for Image to Vector Prediction", 
    "BS": BS,
    "train_data & val_data": "Complete Data",
    "model": model.state_dict(), 
    "optimizer":optimizer.state_dict() ,
    "scheduler": scheduler.state_dict(),
}, "/content/drive/My Drive/EyeGaze/checkpoint_resnet18_acosLoss_25epochs_complete_data.pth", )

NameError: ignored